In [1]:
API_KEY = "sk-API_KEY"

In [ ]:
# 导入sqlite3库
import sqlite3

# 连接到数据库
conn = sqlite3.connect('FlowerShop.db')
cursor = conn.cursor()

# 执行SQL命令来创建Flowers表
cursor.execute('''
        CREATE TABLE Flowers (
            ID INTEGER PRIMARY KEY, 
            Name TEXT NOT NULL, 
            Type TEXT NOT NULL, 
            Source TEXT NOT NULL, 
            PurchasePrice REAL, 
            SalePrice REAL,
            StockQuantity INTEGER, 
            SoldQuantity INTEGER, 
            ExpiryDate DATE,  
            Description TEXT, 
            EntryDate DATE DEFAULT CURRENT_DATE 
        );
    ''')

# 插入5种鲜花的数据
flowers = [
    ('Rose', 'Flower', 'France', 1.2, 2.5, 100, 10, '2023-12-31', 'A beautiful red rose'),
    ('Tulip', 'Flower', 'Netherlands', 0.8, 2.0, 150, 25, '2023-12-31', 'A colorful tulip'),
    ('Lily', 'Flower', 'China', 1.5, 3.0, 80, 5, '2023-12-31', 'An elegant white lily'),
    ('Daisy', 'Flower', 'USA', 0.7, 1.8, 120, 15, '2023-12-31', 'A cheerful daisy flower'),
    ('Orchid', 'Flower', 'Brazil', 2.0, 4.0, 50, 2, '2023-12-31', 'A delicate purple orchid')
]

for flower in flowers:
    cursor.execute('''
        INSERT INTO Flowers (Name, Type, Source, PurchasePrice, SalePrice, StockQuantity, SoldQuantity, ExpiryDate, Description) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    ''', flower)

# 提交更改
conn.commit()

# 关闭数据库连接
conn.close()

In [3]:
# 导入langchain的实用工具和相关的模块
from langchain.utilities import SQLDatabase
from langchain_community.chat_models import ChatTongyi
from langchain_experimental.sql import SQLDatabaseChain

# 连接到FlowerShop数据库（之前我们使用的是Chinook.db）
db = SQLDatabase.from_uri("sqlite:///FlowerShop.db")

# 创建OpenAI的低级语言模型（LLM）实例，这里我们设置温度为0，意味着模型输出会更加确定性
llm = ChatTongyi(model='qwen-max', temperature=0, api_key=API_KEY)
# 创建SQL数据库链实例，它允许我们使用LLM来查询SQL数据库
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# 运行与鲜花运营相关的问题
response = db_chain.run("有多少种不同的鲜花？")
print(response)

response = db_chain.run("哪种鲜花的存货数量最少？")
print(response)

response = db_chain.run("平均销售价格是多少？")
print(response)

response = db_chain.run("从法国进口的鲜花有多少种？")
print(response)

response = db_chain.run("哪种鲜花的销售量最高？")
print(response)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


有多少种不同的鲜花？
SQLQuery:SELECT COUNT(DISTINCT "Name") FROM "Flowers";
SQLResult: [(5,)]
Answer:

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


有5种不同的鲜花。
> Finished chain.
有5种不同的鲜花。
哪种鲜花的存货数量最少？
SQLQuery:SELECT "Name" FROM "Flowers" ORDER BY "StockQuantity" ASC LIMIT 1;
SQLResult: [('Orchid',)]
Answer:

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


存货数量最少的鲜花是Orchid。
> Finished chain.
存货数量最少的鲜花是Orchid。
平均销售价格是多少？
SQLQuery:SELECT AVG("SalePrice") FROM "Flowers"
SQLResult: [(2.66,)]
Answer:

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


平均销售价格是2.66。
> Finished chain.
平均销售价格是2.66。
从法国进口的鲜花有多少种？
SQLQuery:SELECT COUNT(*) FROM "Flowers" WHERE "Source" = 'France';
SQLResult: [(1,)]
Answer:

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


从法国进口的鲜花有1种。
> Finished chain.
从法国进口的鲜花有1种。
哪种鲜花的销售量最高？
SQLQuery:SELECT "Name" FROM "Flowers" ORDER BY "SoldQuantity" DESC LIMIT 1;
SQLResult: [('Tulip',)]
Answer:Tulip的销售量最高。
> Finished chain.
Tulip的销售量最高。
